- recent update: 24.10.29
- update content:
    1. mid-price 생성
    2. significant or insignificant 예측 모델 생성
    3. significant 예측되는 경우에만 significant increase or decrease인지 예측
- target var: mid price return significant change (0 or 1)
- Model: XGBoost(significant or insignificant 예측) + LSTM(significant increase or decrease 예측)

In [43]:
import torch
import torch.nn as nn

class TransformerClassifier(nn.Module):
    def __init__(self, input_dim, num_classes, num_heads=4, num_transformer_blocks=2, d_model=128, dff=256, dropout_rate=0.1, max_seq_length=100):
        super(TransformerClassifier, self).__init__()

        # Linear embedding for input features
        self.embedding = nn.Linear(input_dim, d_model)

        # Positional encoding
        self.positional_encoding = nn.Parameter(torch.zeros(1, max_seq_length, d_model))

        # Transformer encoder blocks
        self.transformer_blocks = nn.ModuleList([
            nn.TransformerEncoderLayer(d_model=d_model, nhead=num_heads, dim_feedforward=dff, dropout=dropout_rate, batch_first=True)
            for _ in range(num_transformer_blocks)
        ])

        # Global average pooling
        self.global_avg_pool = nn.AdaptiveAvgPool1d(1)

        # Fully connected layer for classification
        self.fc = nn.Linear(d_model, num_classes)
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x):
        # Input shape: (batch_size, seq_length, input_dim)

        seq_length = x.size(1)

        # Linear embedding and add positional encoding
        x = self.embedding(x) + self.positional_encoding[:, :seq_length, :]

        # Pass through transformer blocks
        for block in self.transformer_blocks:
            x = block(x)

        # Global average pooling (reduce to (batch_size, d_model))
        x = x.transpose(1, 2)  # For pooling (batch_size, d_model, seq_length)
        x = self.global_avg_pool(x).squeeze(-1)

        # Fully connected layer for classification
        x = self.dropout(x)
        output = self.fc(x)

        return output

# Example usage
input_dim = 80  # Number of features
num_classes = 4  # Number of output classes
batch_size = 64
seq_length = 100

model = TransformerClassifier(input_dim=input_dim, num_classes=num_classes)

# Dummy input: (batch_size, seq_length, input_dim)
dummy_input = torch.randn(batch_size, seq_length, input_dim)

# Forward pass
output = model(dummy_input)
print(output.shape)  # Expected output: (batch_size, num_classes)


torch.Size([64, 4])


In [44]:
# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# Training Loop
for epoch in range(10):  # Number of epochs
    model.train()
    optimizer.zero_grad()

    # Forward pass
    outputs = model(dummy_input)
    targets = torch.randint(0, num_classes, (batch_size,))  # Dummy targets
    loss = criterion(outputs, targets)

    # Backward pass
    loss.backward()
    optimizer.step()

    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")


Epoch 1, Loss: 1.3759
Epoch 2, Loss: 1.3653
Epoch 3, Loss: 1.3867
Epoch 4, Loss: 1.3428
Epoch 5, Loss: 1.4148
Epoch 6, Loss: 1.3874
Epoch 7, Loss: 1.4149
Epoch 8, Loss: 1.4384
Epoch 9, Loss: 1.4079
Epoch 10, Loss: 1.3897


In [45]:
import torch
import torch.nn as nn

class TransformerClassifierNonSequence(nn.Module):
    def __init__(self, input_dim, num_classes, num_heads=4, d_model=128, dff=256, dropout_rate=0.1):
        super(TransformerClassifierNonSequence, self).__init__()

        # Linear embedding to map input_dim to d_model
        self.embedding = nn.Linear(input_dim, d_model)

        # Single Transformer Encoder Layer
        self.self_attention = nn.MultiheadAttention(embed_dim=d_model, num_heads=num_heads, batch_first=True)

        # Feedforward Network
        self.feed_forward = nn.Sequential(
            nn.Linear(d_model, dff),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(dff, d_model)
        )

        # Normalization layers
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)

        # Fully connected output layer
        self.fc = nn.Linear(d_model, num_classes)
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x):
        # Input shape: (batch_size, input_dim)

        # Add a pseudo-sequence dimension (required for MultiheadAttention)
        x = x.unsqueeze(1)  # Shape: (batch_size, 1, input_dim)

        # Linear embedding
        x = self.embedding(x)  # Shape: (batch_size, 1, d_model)

        # Self-attention (requires sequence dimension)
        attention_output, _ = self.self_attention(x, x, x)  # Shape: (batch_size, 1, d_model)
        x = self.norm1(x + attention_output)

        # Feedforward network
        ff_output = self.feed_forward(x)
        x = self.norm2(x + ff_output)  # Shape: (batch_size, 1, d_model)

        # Remove pseudo-sequence dimension
        x = x.squeeze(1)  # Shape: (batch_size, d_model)

        # Fully connected layer for classification
        x = self.dropout(x)
        output = self.fc(x)  # Shape: (batch_size, num_classes)

        return output

# Example usage
input_dim = 80  # Number of features
num_classes = 4  # Number of output classes
batch_size = 64

model = TransformerClassifierNonSequence(input_dim=input_dim, num_classes=num_classes)

# Dummy input: (batch_size, input_dim)
dummy_input = torch.randn(batch_size, input_dim)

# Forward pass
output = model(dummy_input)
print(output.shape)  # Expected output: (batch_size, num_classes)


torch.Size([64, 4])


In [46]:
# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# Training Loop
for epoch in range(10):  # Number of epochs
    model.train()
    optimizer.zero_grad()

    # Forward pass
    outputs = model(dummy_input)
    targets = torch.randint(0, num_classes, (batch_size,))  # Dummy targets
    loss = criterion(outputs, targets)

    # Backward pass
    loss.backward()
    optimizer.step()

    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")


Epoch 1, Loss: 1.4818
Epoch 2, Loss: 1.5601
Epoch 3, Loss: 1.5841
Epoch 4, Loss: 1.4920
Epoch 5, Loss: 1.4566
Epoch 6, Loss: 1.5573
Epoch 7, Loss: 1.4749
Epoch 8, Loss: 1.4216
Epoch 9, Loss: 1.4681
Epoch 10, Loss: 1.4678
